In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json

with open('maestro_articulos_sigcom.json', encoding='utf-8') as file:
    MAESTRO_ARTICULOS = json.load(file)

sns.set_style("darkgrid")
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def obtener_full_path(directorio):
    return [os.path.join(directorio, file) for file in os.listdir(directorio)]

In [3]:
df = pd.concat(map(lambda x: pd.read_csv(x, parse_dates=[
               0], dayfirst=True), obtener_full_path('input')))
df_movimientos = df.sort_values('Fecha')
df_movimientos.columns = df_movimientos.columns.str.replace(' ', '_')
df_movimientos.insert(1, 'Fecha_meses', df_movimientos['Fecha'].dt.to_period('M'))
df_movimientos.insert(2, 'Fecha_dias', df_movimientos['Fecha'].dt.to_period('D'))

In [4]:
df_salidas = df_movimientos.query('Movimiento == "Salida"')
df_salidas_traducidas = df_salidas.copy()
df_salidas_traducidas['Item_SIGFE'] = df_salidas_traducidas['Codigo_Articulo'].apply(
    lambda x: f"{MAESTRO_ARTICULOS[x]['Cod. SIGFE']}-{MAESTRO_ARTICULOS[x]['Item SIGFE']}")


In [9]:
item_presup = df_salidas_traducidas.groupby(by=['Fecha_meses', 'Destino', 'Item_SIGFE', 'Codigo_Articulo+Nombre'])[['Cantidad', 'Neto_Total']].sum().reset_index()

In [7]:
item_presup.to_excel('resumen.xlsx', index=False)

In [10]:
item_presup

,Fecha_meses,Destino,Item_SIGFE,Codigo_Articulo+Nombre,Cantidad,Neto_Total
0,2022-01,ANATOMIA PATOLOGICA,22.04.003.002-Otros Químicos,N103000-HEMATOXYLIN II (05277965001),1,"110,000.00"
1,2022-01,ANATOMIA PATOLOGICA,22.04.003.002-Otros Químicos,N103200-OPTIVIEW AMPLIFICATION KIT (06396518001),1,"850,000.00"
2,2022-01,ANATOMIA PATOLOGICA,22.04.003.002-Otros Químicos,N103300-RABBIT MONOCLONAL NEGATIVE CONTROL IG ...,1,"110,000.00"
3,2022-01,ANATOMIA PATOLOGICA,22.04.003.002-Otros Químicos,N103500-ANTICUERP ANTI ALK (D5F3) (06679072001),1,"1,650,000.00"
4,2022-01,ANATOMIA PATOLOGICA,22.04.005.003-Instrumental Quirúrgico,A030001-ALCOHOL ABSOLUTO FCO 1000CC,12,"174,000.00"
...,...,...,...,...,...,...
21942,2022-11,SECRE. NUTRICION,"22.04.008-Menaje para Oficina, Casino y Otros",ME7103400-CUCHILLO MESA PLASTICO,1000,"25,000.00"
21943,2022-11,SECRE. NUTRICION,"22.04.008-Menaje para Oficina, Casino y Otros",ME7110900-VASO TERMICO,1000,"50,000.00"
21944,2022-11,SECRE. NUTRICION,"22.04.008-Menaje para Oficina, Casino y Otros",ME712100-CONTENEDOR ESTUCHE 161 PET,1600,"81,600.00"
21945,2022-11,SECRE. NUTRICION,"22.04.008-Menaje para Oficina, Casino y Otros",ME7122000-BANDEJA TERMICA COLACION 2 DIVISION,200,"42,000.00"
